In [43]:
import pandas as pd
import numpy as np

In [44]:
# Load the dataset
df = pd.read_csv('all_kindle_review.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [45]:
df = df[['reviewText','rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [46]:
df.shape

(12000, 2)

In [47]:
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [48]:
df.nunique()

reviewText    12000
rating            5
dtype: int64

In [49]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

#### Preprocessing and cleaning

In [50]:
## Puting all reviews below 3 as 0 (negative) 
## and reviews above or equal to 3 as 1 (positive)

df['rating'] = df['rating'].apply(lambda x: 0 if x<3 else 1)

In [51]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [52]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [53]:
## Lower case
df['reviewText']=df['reviewText'].str.lower()

In [54]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [55]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Welcome\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [56]:
## Removing stopwords , punctuautions n otehr unnecessary things

## Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url 
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
df['reviewText'] = df['reviewText'].astype(str).apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split())) 

In [57]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [58]:
## Lemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [59]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [60]:
df['reviewText'] = df['reviewText'].apply(lambda x: lemmatize_words(x))

In [61]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [69]:
## Train test split
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(df['reviewText'],df['rating'],test_size = 0.2)

In [71]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()
X_train_bow = bow.fit_transform(X_train).toarray()
X_test_bow = bow.transform(X_test).toarray()

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [73]:
from sklearn.naive_bayes import GaussianNB

nb_model_bow = GaussianNB().fit(X_train_bow,y_train)
nb_model_tfidf = GaussianNB().fit(X_train_tfidf,y_train)

In [74]:
from sklearn.metrics import confusion_matrix , classification_report,accuracy_score

In [75]:
y_pred_bow = nb_model_bow.predict(X_test_bow)
y_pred_tfidf = nb_model_tfidf.predict(X_test_tfidf)

In [78]:
print('BOW accuracy: ' , accuracy_score(y_test,y_pred_bow))
print('TF-IDF accuracy: ' , accuracy_score(y_test,y_pred_tfidf))

BOW accuracy:  0.58625
TF-IDF accuracy:  0.5854166666666667


#### getting very less accuracy , now lets try avg word2vec

In [79]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from tqdm import tqdm
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [80]:
X_train_tokens = [simple_preprocess(doc) for doc in X_train.astype(str)]
X_test_tokens = [simple_preprocess(doc) for doc in X_test.astype(str)]

In [ ]:
# Train Word2Vec on training tokens
w2v = Word2Vec(sentences=X_train_tokens)

In [82]:
# Helper: get average vector for one token list
def doc_vector_avg(tokens, model):
    vecs = [model.wv[word] for word in tokens if word in model.wv]
    if not vecs:
        return np.zeros(model.vector_size)
    return np.mean(vecs, axis=0)

In [83]:
X_train_w2v = np.vstack([doc_vector_avg(t, w2v) for t in tqdm(X_train_tokens, desc='build train w2v')])
X_test_w2v  = np.vstack([doc_vector_avg(t, w2v) for t in tqdm(X_test_tokens, desc='build test w2v')])


build test w2v: 100%|██████████| 2400/2400 [00:00<00:00, 15649.80it/s]


In [84]:
# Train classifier on averaged Word2Vec vectors
clf = LogisticRegression(max_iter=1000, random_state=42)
clf.fit(X_train_w2v, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [85]:
y_pred_w2v = clf.predict(X_test_w2v)
print('Word2Vec (avg) accuracy:', accuracy_score(y_test, y_pred_w2v))
print(classification_report(y_test, y_pred_w2v))


Word2Vec (avg) accuracy: 0.75375
              precision    recall  f1-score   support

           0       0.68      0.53      0.60       824
           1       0.78      0.87      0.82      1576

    accuracy                           0.75      2400
   macro avg       0.73      0.70      0.71      2400
weighted avg       0.75      0.75      0.75      2400



#### using pretrained google word2vec

In [87]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [88]:
X_train_tokens = [simple_preprocess(doc, deacc=True) for doc in X_train.astype(str)]
X_test_tokens  = [simple_preprocess(doc, deacc=True)  for doc in X_test.astype(str)]

In [89]:
def doc_vector_avg(tokens, wv_model):
    vecs = [wv_model[word] for word in tokens if word in wv_model.key_to_index]
    if not vecs:
        return np.zeros(wv_model.vector_size)
    return np.mean(vecs, axis=0)

In [90]:
X_train_w2v = np.vstack([doc_vector_avg(t, wv) for t in tqdm(X_train_tokens, desc='build train w2v')])
X_test_w2v  = np.vstack([doc_vector_avg(t, wv) for t in tqdm(X_test_tokens, desc='build test w2v')])

build test w2v: 100%|██████████| 2400/2400 [00:00<00:00, 15587.02it/s]


In [91]:
clf = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
clf.fit(X_train_w2v, y_train)
y_pred_w2v = clf.predict(X_test_w2v)
print('Word2Vec (GoogleNews avg) accuracy:', accuracy_score(y_test, y_pred_w2v))
print(classification_report(y_test, y_pred_w2v))

Word2Vec (GoogleNews avg) accuracy: 0.8033333333333333
              precision    recall  f1-score   support

           0       0.68      0.80      0.74       824
           1       0.89      0.80      0.84      1576

    accuracy                           0.80      2400
   macro avg       0.78      0.80      0.79      2400
weighted avg       0.82      0.80      0.81      2400

